In [1]:
from pyomo.environ import *

## An Easy Two-stage Stochastic Programming Example in Process Design


<img src="illustrations/so_reaction.png" alt="process" width="800" height="500"/>


### Nomenclature:

#### Decision Variables:

- $F_{A}$: flow rate of A (reactant)

- $F_{B}$: flow rate of B (reactant)

- $F_{C}$: flow rate of C (product)

- $F_{D}$: flow rate of D (product)

- $R_{C}$: backorder C

- $R_{C}$: backorder D

- $y_1$: whether reaction 1 is selected.

- $y_2$: whether reaction 2 is selected.

#### Uncertain Parameters:

- $P_{C}, P_{D}$: Market prices for C,D at the end of the season (exogenous uncertainty).

- $D_{C}, D_{D}$: Demand for C, D

#### Deterministic parameters

- $\xi_{1}, \xi_{2}$: Yields for reaction 1 and 2

- $C^{v}_{A}, C^{v}_{B}$: unit cost of A and B, respectively.

- $C^{f}_{1}, C^{f}_{2}$: fixed cost of reactor 1 and 2, respectively.

- $Q$: Processing capacity for the reactor.

- $P^{B}_{C}, P^{B}_{D}$: Backorder price for C and D.

### Deterministic Formulation

#### Constraints:

1 **Capacity Constraints**:
\begin{align*}
F_{A, t} \le Q \cdot y_{1, t} \\
F_{B, t} \le Q \cdot y_{2, t}
\end{align*}

2 **Unit Constraints**:
\begin{align*}
y_{1, 2} - y_{1, 1}  \le Ucap \\
y_{2, 2} - y_{2, 1}  \le Ucap \\
y_{1, 1} \le 1,  y_{2, 1} \le 1\\
y_{1, t} \ge 0,  y_{2, t} \ge 0
\end{align*}

3 **Yield Constraints**:

\begin{align*}
\xi_{1} F_{A,t} = F_{C,t} \\
\xi_{2} F_{B,t} = F_{D,t}
\end{align*}

4 **

/backorder Constraint**:

\begin{align*}
F_{C,t} + R_{C,t} >= D_{C,t} \\
F_{D,t} + R_{D,t} >= D_{D,t}
\end{align*}

#### Objective:

$$ \textbf{min} \quad \sum_{t} (C^{f}_{1} y_{1,t} + C^{f}_{2}y_{2,t}) + (C^{v}_{A}F_{A,t} + C^{v}_{B}F_{B,t}) + ( P^{B}_{C}R_{C,t} + P^{B}_{D} R_{D,t} - P_{C}F_{C,t} - P_{D}F_{D,t})  $$

In [2]:
def build_det():
    m = ConcreteModel()

    m.t = Set(initialize=[1,2], doc="period")

    m.Q = Param(initialize=18)
    m.Ucap = Param(initialize=1)
    m.xi1 = Param(initialize=0.8)
    m.xi2 = Param(initialize=0.5)
    m.cf1 = Param(initialize=20)
    m.cf2 = Param(initialize=50)
    m.cva = Param(initialize=2)
    m.cvb = Param(initialize=4)
    m.pbc = Param(initialize=20)
    m.pbd = Param(initialize=25)

    m.pc = Param(m.t, initialize=10)
    m.pd = Param(m.t, initialize=12)
    m.Dc = Param(m.t, initialize=16)
    m.Dd = Param(m.t, initialize=2)

    m.Fa = Var(m.t, domain=NonNegativeReals)
    m.Fb = Var(m.t, domain=NonNegativeReals)
    m.Fc = Var(m.t, domain=NonNegativeReals)
    m.Fd = Var(m.t, domain=NonNegativeReals)
    m.Rc = Var(m.t, domain=NonNegativeReals)
    m.Rd = Var(m.t, domain=NonNegativeReals)

    m.y1 = Var(m.t, domain=Integers)
    m.y2 = Var(m.t, domain=Integers)

    def _c1a(m, t):
        return m.Fa[t] <= m.Q * m.y1[t]
    m.c1a = Constraint(m.t, rule=_c1a)

    def _c1b(m, t):
        return m.Fb[t] <= m.Q * m.y2[t]
    m.c1b = Constraint(m.t, rule=_c1b)

    def _c2a(m, t):
        if t == m.t.first():
            return Constraint.Skip
        return m.y1[t] - m.y1[t-1] <= m.Ucap
    m.c2a = Constraint(m.t, rule=_c2a)

    def _c2b(m, t):
        if t == m.t.first():
            return Constraint.Skip
        return m.y2[t] - m.y2[t-1] <= m.Ucap
    m.c2b = Constraint(m.t, rule=_c2b)

    m.y1[1].setub(1)
    m.y2[1].setub(1)
    m.y1.setlb(0)
    m.y2.setlb(0)

    def _c3a(m, t):
        return m.xi1 * m.Fa[t] == m.Fc[t]
    m.c3a = Constraint(m.t, rule=_c3a)

    def _c3b(m, t):
        return m.xi2 * m.Fb[t] == m.Fd[t]
    m.c3b = Constraint(m.t, rule=_c3b)

    def _c4c(m, t):
        return m.Fc[t] + m.Rc[t] >= m.Dc[t]
    m.c4c = Constraint(m.t, rule=_c4c)

    def _c4d(m, t):
        return m.Fd[t] + m.Rd[t] >= m.Dd[t]
    m.c4d = Constraint(m.t, rule=_c4d)

    m.obj = Objective(
        expr = sum((m.cf1*m.y1[t] + m.cf2*m.y2[t]) + (m.cva*m.Fa[t] + m.cvb*m.Fb[t])
                   + (m.pbc*m.Rc[t] + m.pbd*m.Rd[t] - m.pc[t]*(m.Fc[t]+m.Rc[t]) - m.pd[t]*(m.Fd[t]+m.Rd[t]))
                   for t in m.t),
        sense=minimize
    )

    return m



In [3]:
mdet = build_det()

opt1 = SolverFactory('gurobi_persistent')
opt1.set_instance(mdet)
opt1.solve();

Restricted license - for non-production use only - expires 2025-11-24


In [4]:
for v in mdet.component_objects(Var, active=True):
    v.pprint()
value(mdet.obj)

Fa : Size=2, Index=t
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  18.0 :  None : False : False : NonNegativeReals
      2 :     0 :  36.0 :  None : False : False : NonNegativeReals
Fb : Size=2, Index=t
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  18.0 :  None : False : False : NonNegativeReals
      2 :     0 :  18.0 :  None : False : False : NonNegativeReals
Fc : Size=2, Index=t
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  14.4 :  None : False : False : NonNegativeReals
      2 :     0 :  28.8 :  None : False : False : NonNegativeReals
Fd : Size=2, Index=t
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   9.0 :  None : False : False : NonNegativeReals
      2 :     0 :   9.0 :  None : False : False : NonNegativeReals
Rc : Size=2, Index=t
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.6 :  None : False : False : NonNegativeReals
      2

-220.0

### Stochastic formulation

The first period is deterministic, and the second period is uncertain with three scenarios with equal probability $\Psi(\omega)$.

Variables

Group 1: discrete
$$ y^{\omega}_{1,t}, y^{\omega}_{2,t} $$

Group 2: continuous
$$ F^{\omega}_{A,t}, F^{\omega}_{B,t}, F^{\omega}_{C,t}, F^{\omega}_{D,t} $$
$$ R^{\omega}_{C,t}, R^{\omega}_{D,t} $$

#### Constraints:

1 **Capacity Constraints**:
\begin{align*}
F^{\omega}_{A, t} \le Q \cdot y^{\omega}_{1, t} \\
F^{\omega}_{B, t} \le Q \cdot y^{\omega}_{2, t}
\end{align*}

2 **Unit Constraints**:
\begin{align*}
y^{\omega}_{1, 2} - y^{\omega}_{1, 1}  \le Ucap \\
y^{\omega}_{2, 2} - y^{\omega}_{2, 1}  \le Ucap \\
y^{\omega}_{1, 1} \le 1,  y^{\omega}_{2, 1} \le 1\\
y^{\omega}_{1, t} \ge 0,  y^{\omega}_{2, t} \ge 0
\end{align*}

3 **Yield Constraints**:

\begin{align*}
\xi_{1} F^{\omega}_{A,t} = F^{\omega}_{C,t} \\
\xi_{2} F^{\omega}_{B,t} = F^{\omega}_{D,t}
\end{align*}

4 **Demand/backorder Constraint**:

\begin{align*}
F^{\omega}_{C,t} + R^{\omega}_{C,t} >= D^{\omega}_{C,t} \\
F^{\omega}_{D,t} + R^{\omega}_{D,t} >= D^{\omega}_{D,t}
\end{align*}

5 **NACs**:

\begin{align*}
    x^{\omega}_{1} = x^{1}_{1}
\end{align*}

#### Objective:

$$ \textbf{min} \quad \sum_{\omega} \Psi^{\omega} \sum_{t} (C^{f}_{1} y^{\omega}_{1,t} + C^{f}_{2}y^{\omega}_{2,t}) + (C^{v}_{A}F^{\omega}_{A,t} + C^{v}_{B}F^{\omega}_{B,t}) + ( P^{B}_{C}R^{\omega}_{C,t} + P^{B}_{D} R^{\omega}_{D,t} - P^{\omega}_{C}F^{\omega}_{C,t} - P^{\omega}_{D}F^{\omega}_{D,t})  $$


In [5]:
def build_sp():
    m = ConcreteModel()

    m.t = Set(initialize=[1,2], doc="period")
    m.omega = Set(initialize=[1,2,3], doc="scenarios")

    # Probability of scenario omega
    m.Psi = Param(m.omega, initialize=1/3)

    # deterministic parameter
    m.Q = Param(initialize=18)
    m.Ucap = Param(initialize=1)
    m.xi1 = Param(initialize=0.8)
    m.xi2 = Param(initialize=0.5)
    m.cf1 = Param(initialize=20)
    m.cf2 = Param(initialize=50)
    m.cva = Param(initialize=2)
    m.cvb = Param(initialize=4)
    m.pbc = Param(initialize=20)
    m.pbd = Param(initialize=25)

    # uncertain parameter
    pc_init = {}
    for o in m.omega:
        pc_init[1,o] = 10
    pc_init[2,1] = 5
    pc_init[2,2] = 10
    pc_init[2,3] = 15
    m.pc = Param(m.t, m.omega, initialize=pc_init)

    pd_init = {}
    for o in m.omega:
        pd_init[1,o] = 12
    pd_init[2,1] = 6
    pd_init[2,2] = 12
    pd_init[2,3] = 18
    m.pd = Param(m.t, m.omega, initialize=pd_init)

    Dc_init = {}
    for o in m.omega:
        Dc_init[1,o] = 16
    Dc_init[2,1] = 10
    Dc_init[2,2] = 16
    Dc_init[2,3] = 25
    m.Dc = Param(m.t, m.omega, initialize=Dc_init)

    Dd_init = {}
    for o in m.omega:
        Dd_init[1,o] = 2
    Dd_init[2,1] = 0
    Dd_init[2,2] = 2
    Dd_init[2,3] = 16
    m.Dd = Param(m.t, m.omega, initialize=Dd_init)

    # continuous variables
    m.Fa = Var(m.t, m.omega, domain=NonNegativeReals)
    m.Fb = Var(m.t, m.omega, domain=NonNegativeReals)
    m.Fc = Var(m.t, m.omega, domain=NonNegativeReals)
    m.Fd = Var(m.t, m.omega, domain=NonNegativeReals)
    m.Rc = Var(m.t, m.omega, domain=NonNegativeReals)
    m.Rd = Var(m.t, m.omega, domain=NonNegativeReals)

    # discrete variables
    m.y1 = Var(m.t, m.omega, domain=Integers)
    m.y2 = Var(m.t, m.omega, domain=Integers)

    def _c1a(m, t, o):
        return m.Fa[t, o] <= m.Q * m.y1[t,o]
    m.c1a = Constraint(m.t, m.omega, rule=_c1a)

    def _c1b(m, t, o):
        return m.Fb[t, o] <= m.Q * m.y2[t, o]
    m.c1b = Constraint(m.t, m.omega, rule=_c1b)

    def _c2a(m, t, o):
        if t == m.t.first():
            return Constraint.Skip
        return m.y1[t, o] - m.y1[t-1, o] <= m.Ucap
    m.c2a = Constraint(m.t, m.omega, rule=_c2a)

    def _c2b(m, t, o):
        if t == m.t.first():
            return Constraint.Skip
        return m.y2[t, o] - m.y2[t-1, o] <= m.Ucap
    m.c2b = Constraint(m.t, m.omega, rule=_c2b)

    for o in m.omega:
        m.y1[1, o].setub(1)
        m.y2[1, o].setub(1)
    m.y1.setlb(0)
    m.y2.setlb(0)

    def _c3a(m, t, o):
        return m.xi1 * m.Fa[t, o] == m.Fc[t, o]
    m.c3a = Constraint(m.t, m.omega, rule=_c3a)

    def _c3b(m, t, o):
        return m.xi2 * m.Fb[t, o] == m.Fd[t, o]
    m.c3b = Constraint(m.t, m.omega, rule=_c3b)

    def _c4c(m, t, o):
        return m.Fc[t, o] + m.Rc[t, o] >= m.Dc[t, o]
    m.c4c = Constraint(m.t, m.omega, rule=_c4c)

    def _c4d(m, t, o):
        return m.Fd[t, o] + m.Rd[t, o] >= m.Dd[t, o]
    m.c4d = Constraint(m.t, m.omega, rule=_c4d)

    def nac_y1(m, t, o):
        if not t == m.omega.first():
            return Constraint.Skip
        if o == m.omega.first():
            return Constraint.Skip
        return m.y1[t,o] == m.y1[t,1]
    m.nac_y1 = Constraint(m.t, m.omega, rule=nac_y1)

    def nac_y2(m, t, o):
        if not t == m.omega.first():
            return Constraint.Skip
        if o == m.omega.first():
            return Constraint.Skip
        return m.y2[t,o] == m.y2[t,1]
    m.nac_y2 = Constraint(m.t, m.omega, rule=nac_y2)

    def nac_Fa(m, t, o):
        if not t == m.omega.first():
            return Constraint.Skip
        if o == m.omega.first():
            return Constraint.Skip
        return m.Fa[t,o] == m.Fa[t,1]
    m.nac_Fa = Constraint(m.t, m.omega, rule=nac_Fa)

    def nac_Fb(m, t, o):
        if not t == m.omega.first():
            return Constraint.Skip
        if o == m.omega.first():
            return Constraint.Skip
        return m.Fb[t,o] == m.Fb[t,1]
    m.nac_Fb = Constraint(m.t, m.omega, rule=nac_Fb)

    def nac_Fc(m, t, o):
        if not t == m.omega.first():
            return Constraint.Skip
        if o == m.omega.first():
            return Constraint.Skip
        return m.Fc[t,o] == m.Fc[t,1]
    m.nac_Fc = Constraint(m.t, m.omega, rule=nac_Fc)

    def nac_Fd(m, t, o):
        if not t == m.omega.first():
            return Constraint.Skip
        if o == m.omega.first():
            return Constraint.Skip
        return m.Fd[t,o] == m.Fd[t,1]
    m.nac_Fd = Constraint(m.t, m.omega, rule=nac_Fd)

    def nac_Rc(m, t, o):
        if not t == m.omega.first():
            return Constraint.Skip
        if o == m.omega.first():
            return Constraint.Skip
        return m.Rc[t,o] == m.Rc[t,1]
    m.nac_Rc = Constraint(m.t, m.omega, rule=nac_Rc)

    def nac_Rd(m, t, o):
        if not t == m.omega.first():
            return Constraint.Skip
        if o == m.omega.first():
            return Constraint.Skip
        return m.Rd[t,o] == m.Rd[t,1]
    m.nac_Rd = Constraint(m.t, m.omega, rule=nac_Rd)

    m.obj = Objective(
        expr = sum(
            m.Psi[omega] * sum(
                (m.cf1*m.y1[t, omega] + m.cf2*m.y2[t, omega]) + (m.cva*m.Fa[t, omega] + m.cvb*m.Fb[t, omega])
                + (m.pbc*m.Rc[t, omega] +m.pbd*m.Rd[t, omega]
                   - m.pc[t, omega]*(m.Fc[t, omega]+m.Rc[t, omega]) - m.pd[t, omega]*(m.Fd[t, omega]+m.Rd[t, omega]))
                for t in m.t)
            for omega in m.omega),
        sense=minimize
    )

    return m

In [6]:
msp = build_sp()

opt2 = SolverFactory('gurobi_persistent')
opt2.set_instance(msp)
opt2.solve();

In [7]:
for v in msp.component_objects(Var, active=True):
    v.pprint()
value(msp.obj)

Fa : Size=6, Index=Fa_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (1, 2) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (1, 3) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 :  36.0 :  None : False : False : NonNegativeReals
    (2, 2) :     0 :  36.0 :  None : False : False : NonNegativeReals
    (2, 3) :     0 :  36.0 :  None : False : False : NonNegativeReals
Fb : Size=6, Index=Fb_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (1, 2) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (1, 3) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 2) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (2, 3) :     0 :  36.0 :  None : False : False : N

-256.0

In [8]:
def build_sp2():
    m = ConcreteModel()

    m.t = Set(initialize=[1,2], doc="period")
    m.omega = Set(initialize=[1,2,3], doc="scenarios")

    # Probability of scenario omega
    m.Psi = Param(m.omega, initialize=1/3)

    # deterministic parameter
    m.Q = Param(initialize=18)
    m.Ucap = Param(initialize=1)
    m.xi1 = Param(initialize=0.8)
    m.xi2 = Param(initialize=0.5)
    m.cf1 = Param(initialize=20)
    m.cf2 = Param(initialize=50)
    m.cva = Param(initialize=2)
    m.cvb = Param(initialize=4)
    m.pbc = Param(initialize=20)
    m.pbd = Param(initialize=25)

    # uncertain parameter
    pc_init = {}
    for o in m.omega:
        pc_init[1,o] = 10
    pc_init[2,1] = 5
    pc_init[2,2] = 10
    pc_init[2,3] = 15
    m.pc = Param(m.t, m.omega, initialize=pc_init)

    pd_init = {}
    for o in m.omega:
        pd_init[1,o] = 12
    pd_init[2,1] = 6
    pd_init[2,2] = 12
    pd_init[2,3] = 18
    m.pd = Param(m.t, m.omega, initialize=pd_init)

    Dc_init = {}
    for o in m.omega:
        Dc_init[1,o] = 16
    Dc_init[2,1] = 10
    Dc_init[2,2] = 16
    Dc_init[2,3] = 25
    m.Dc = Param(m.t, m.omega, initialize=Dc_init)

    Dd_init = {}
    for o in m.omega:
        Dd_init[1,o] = 2
    Dd_init[2,1] = 0
    Dd_init[2,2] = 2
    Dd_init[2,3] = 16
    m.Dd = Param(m.t, m.omega, initialize=Dd_init)

    # continuous variables
    m.Fa = Var(m.t, m.omega, domain=NonNegativeReals)
    m.Fb = Var(m.t, m.omega, domain=NonNegativeReals)
    m.Fc = Var(m.t, m.omega, domain=NonNegativeReals)
    m.Fd = Var(m.t, m.omega, domain=NonNegativeReals)
    m.Rc = Var(m.t, m.omega, domain=NonNegativeReals)
    m.Rd = Var(m.t, m.omega, domain=NonNegativeReals)

    # discrete variables
    m.y1 = Var(m.t, m.omega, domain=Integers)
    m.y2 = Var(m.t, m.omega, domain=Integers)

    def _c1a(m, t, o):
        return m.Fa[t, o] <= m.Q * m.y1[t,o]
    m.c1a = Constraint(m.t, m.omega, rule=_c1a)

    def _c1b(m, t, o):
        return m.Fb[t, o] <= m.Q * m.y2[t, o]
    m.c1b = Constraint(m.t, m.omega, rule=_c1b)

    def _c2a(m, t, o):
        if t == m.t.first():
            return Constraint.Skip
        return m.y1[t, o] - m.y1[t-1, o] <= m.Ucap
    m.c2a = Constraint(m.t, m.omega, rule=_c2a)

    def _c2b(m, t, o):
        if t == m.t.first():
            return Constraint.Skip
        return m.y2[t, o] - m.y2[t-1, o] <= m.Ucap
    m.c2b = Constraint(m.t, m.omega, rule=_c2b)

    for o in m.omega:
        m.y1[1, o].setub(1)
        m.y2[1, o].setub(1)
    m.y1.setlb(0)
    m.y2.setlb(0)

    def _c3a(m, t, o):
        return m.xi1 * m.Fa[t, o] == m.Fc[t, o]
    m.c3a = Constraint(m.t, m.omega, rule=_c3a)

    def _c3b(m, t, o):
        return m.xi2 * m.Fb[t, o] == m.Fd[t, o]
    m.c3b = Constraint(m.t, m.omega, rule=_c3b)

    def _c4c(m, t, o):
        return m.Fc[t, o] + m.Rc[t, o] >= m.Dc[t, o]
    m.c4c = Constraint(m.t, m.omega, rule=_c4c)

    def _c4d(m, t, o):
        return m.Fd[t, o] + m.Rd[t, o] >= m.Dd[t, o]
    m.c4d = Constraint(m.t, m.omega, rule=_c4d)

    def nac_y1(m, t, o):
        if not t == m.omega.first():
            return Constraint.Skip
        if o == m.omega.first():
            return Constraint.Skip
        return m.y1[t,o] == m.y1[t,1]
    m.nac_y1 = Constraint(m.t, m.omega, rule=nac_y1)

    def nac_y2(m, t, o):
        if not t == m.omega.first():
            return Constraint.Skip
        if o == m.omega.first():
            return Constraint.Skip
        return m.y2[t,o] == m.y2[t,1]
    m.nac_y2 = Constraint(m.t, m.omega, rule=nac_y2)

    m.obj = Objective(
        expr = sum(
            m.Psi[omega] * sum(
                (m.cf1*m.y1[t, omega] + m.cf2*m.y2[t, omega]) + (m.cva*m.Fa[t, omega] + m.cvb*m.Fb[t, omega])
                + (m.pbc*m.Rc[t, omega] +m.pbd*m.Rd[t, omega]
                   - m.pc[t, omega]*(m.Fc[t, omega]+m.Rc[t, omega]) - m.pd[t, omega]*(m.Fd[t, omega]+m.Rd[t, omega]))
                for t in m.t)
            for omega in m.omega),
        sense=minimize
    )

    return m

In [9]:
msp2 = build_sp2()

opt3 = SolverFactory('gurobi_persistent')
opt3.set_instance(msp2)
opt3.solve();

In [10]:
for v in msp2.component_objects(Var, active=True):
    v.pprint()
value(msp2.obj)

Fa : Size=6, Index=Fa_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (1, 2) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (1, 3) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 :  36.0 :  None : False : False : NonNegativeReals
    (2, 2) :     0 :  36.0 :  None : False : False : NonNegativeReals
    (2, 3) :     0 :  36.0 :  None : False : False : NonNegativeReals
Fb : Size=6, Index=Fb_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (1, 2) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (1, 3) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 2) :     0 :  18.0 :  None : False : False : NonNegativeReals
    (2, 3) :     0 :  36.0 :  None : False : False : N

-256.0